### this notebook saves the metrics (static ones; and lss ones) for a few selected v1.7 dbs.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import os
import pandas as pd
%matplotlib inline

In [2]:
data_dir = '/global/cscratch1/sd/awan/lsst_output/post_wp_output_v1.7_-0.1cuts/summary_data/'
outdir = '/global/homes/a/awan/LSST/lsstRepos/ObsStrat/postwp/paper-data/summary_csv_v1.7_-0.1cuts/'
os.makedirs(outdir, exist_ok=True)

#### read in the data for the static metrics and store as a dictionary

In [3]:
# read in the data 
data = {}
yr_label = {}
for yr in [1, 3, 6, 10]:
    files = [f for f in os.listdir( data_dir ) if f.endswith('csv') and f.__contains__('y%s_' % yr) and f.startswith('eg_') ]
    for file in files:
        print( file )
        key = 'yr%s_%s' % (yr, file.split('_')[4])
        data[ key ] = pd.read_csv('%s/%s' % (data_dir, file))
        yr_label[ key ] =  r'Y%s (i$>$%s) ' % (yr, file.split('_')[4].split('limi')[-1])
        
print( yr_label )
print(data.keys())
print(data[list(data.keys())[0]].keys() )

eg_footprint_stats_y1_limi24.65_nodither_nside256.csv
eg_footprint_stats_y3_limi25.25_nodither_nside256.csv
eg_footprint_stats_y6_limi25.619999999999997_nodither_nside256.csv
eg_footprint_stats_y10_limi25.9_nodither_nside256.csv
{'yr1_limi24.65': 'Y1 (i$>$24.65) ', 'yr3_limi25.25': 'Y3 (i$>$25.25) ', 'yr6_limi25.619999999999997': 'Y6 (i$>$25.619999999999997) ', 'yr10_limi25.9': 'Y10 (i$>$25.9) '}
dict_keys(['yr1_limi24.65', 'yr3_limi25.25', 'yr6_limi25.619999999999997', 'yr10_limi25.9'])
Index(['dbname', 'Area (deg2)', '$i$-band depth: median',
       ' $i$-band depth: std'],
      dtype='object')


In [4]:
# read in the static metrics
for key in data:
    yr_tag = key.split('_')[0].split('yr')[-1]
    lim_tag = key.split('_')[1].split('limi')[-1]
    print(yr_tag, lim_tag)

    data[key] = data[key].rename(columns={'Area (deg2)': 'Y%s effective survey area ' % (yr_tag),
                                          '$i$-band depth: median': 'median Y%s $i$-band coadded depth in effective survey area' % (yr_tag),
                                          ' $i$-band depth: std': 'std in Y%s $i$-band coadded depth in effective survey area' % (yr_tag)} )
    #fname = 'lss_metrics_%s.csv' % key
    #data[key].to_csv('%s/%s' % (outdir, fname), index=False)

1 24.65
3 25.25
6 25.619999999999997
10 25.9


#### get the order of the dbs

In [5]:
data['yr10_limi25.9'].dbname.values

array(['twi_pairs_repeat_v1.7_10yrs', 'twi_neo_pattern6_v1.7_10yrs',
       'rolling_nm_scale0.4_nslice2_v1.7_10yrs', 'footprint_1_v1.710yrs',
       'footprint_6_v1.710yrs', 'twi_neo_pattern5_v1.7_10yrs',
       'rolling_scale0.9_nslice3_v1.7_10yrs', 'euclid_dither2_v1.7_10yrs',
       'euclid_dither4_v1.7_10yrs', 'euclid_dither1_v1.7_10yrs',
       'euclid_dither5_v1.7_10yrs', 'euclid_dither3_v1.7_10yrs',
       'pair_times_33_v1.7_10yrs', 'footprint_4_v1.710yrs',
       'footprint_7_v1.710yrs', 'pair_times_44_v1.7_10yrs',
       'rolling_scale0.2_nslice3_v1.7_10yrs', 'footprint_0_v1.710yrs',
       'rolling_scale1.0_nslice3_v1.7_10yrs',
       'rolling_nm_scale0.4_nslice3_v1.7_10yrs',
       'u_long_ms_30_v1.7_10yrs', 'rolling_scale0.9_nslice2_v1.7_10yrs',
       'rolling_scale0.6_nslice3_v1.7_10yrs',
       'rolling_scale0.6_nslice2_v1.7_10yrs',
       'cadence_drive_gl200v1.7_10yrs',
       'rolling_nm_scale0.9_nslice2_v1.7_10yrs', 'footprint_2_v1.710yrs',
       'rolling_nm_scale

In [6]:
# need to do this since input order is provided for only some of the dbs
df = pd.DataFrame(['baseline_nexp1_v1.7_10yrs',
                   'baseline_nexp2_v1.7_10yrs',
                   'u_long_ms_50_v1.7_10yrs',
                   'rolling_scale0.2_nslice2_v1.7_10yrs',
                   'rolling_scale1.0_nslice2_v1.7_10yrs',
                   'footprint_8_v1.710yrs',
                  ], columns=['name'])
df.to_csv('%s/given_order_v1.7_selected.csv' % outdir, index=False)

In [7]:
df_in_order = pd.read_csv('%s/given_order_v1.7_selected.csv' % outdir, )
df_in_order

,name
0,baseline_nexp1_v1.7_10yrs
1,baseline_nexp2_v1.7_10yrs
2,u_long_ms_50_v1.7_10yrs
3,rolling_scale0.2_nslice2_v1.7_10yrs
4,rolling_scale1.0_nslice2_v1.7_10yrs
5,footprint_8_v1.710yrs


In [8]:
# check some things:
len( list( df_in_order['name'] ) ), len( np.unique( df_in_order['name'] ) )

(6, 6)

In [9]:
yr_key = [f for f in data.keys() if f.__contains__('yr1_')][0]
yr_key

'yr1_limi24.65'

#### find the indices to get the order

In [10]:
# find the indices to get the order
ind = []
for db in df_in_order['name']:
    ind_temp = np.where( data[yr_key]['dbname'].values == db.split('.db')[0] )[0]

    if len(ind_temp) == 0:
        print('%s not found in data dict' % db)
        #ind_temp = np.array([ 0 ])
        
    if ind_temp in ind:
        print('%s ind already in. %s' % (ind_temp, db))
    ind +=  list( ind_temp )
    
ind = np.array(ind)
print(ind)

[33 69 68 36 70 52]


In [11]:
# check we have the right indices
print( data[yr_key]['dbname'].values[28],  df_in_order['name'][0] )

# check we have the right number of inds
print( len(ind), len(np.unique(ind)) )

twi_pairs_mixed_v1.7_10yrs baseline_nexp1_v1.7_10yrs
6 6


#### save the data in the order in which to save things

In [12]:
data.keys()

dict_keys(['yr1_limi24.65', 'yr3_limi25.25', 'yr6_limi25.619999999999997', 'yr10_limi25.9'])

In [13]:
for yr_lim_tag in data:
    for colname in data[yr_lim_tag]:
        if colname != 'dbname':
             df_in_order[colname] = data[yr_lim_tag][colname].values[ind]

In [14]:
df_in_order

,name,Y1 effective survey area,median Y1 $i$-band coadded depth in effective survey area,std in Y1 $i$-band coadded depth in effective survey area,Y3 effective survey area,median Y3 $i$-band coadded depth in effective survey area,std in Y3 $i$-band coadded depth in effective survey area,Y6 effective survey area,median Y6 $i$-band coadded depth in effective survey area,std in Y6 $i$-band coadded depth in effective survey area,Y10 effective survey area,median Y10 $i$-band coadded depth in effective survey area,std in Y10 $i$-band coadded depth in effective survey area
0,baseline_nexp1_v1.7_10yrs,15205.27,25.10,0.18,15383.52,25.73,0.14,15312.39,26.07,0.13,15307.98,26.33,0.12
1,baseline_nexp2_v1.7_10yrs,14765.22,25.03,0.17,15275.67,25.66,0.13,15197.30,26.00,0.12,15179.99,26.26,0.11
2,u_long_ms_50_v1.7_10yrs,14369.76,25.01,0.16,15202.39,25.63,0.12,15129.26,25.97,0.12,15091.39,26.24,0.11
3,rolling_scale0.2_nslice2_v1.7_10yrs,14779.75,25.03,0.17,15303.84,25.66,0.13,15207.11,26.00,0.12,15191.37,26.26,0.12
4,rolling_scale1.0_nslice2_v1.7_10yrs,14779.75,25.03,0.17,15253.38,25.64,0.13,15126.22,25.97,0.12,15100.41,26.24,0.11
5,footprint_8_v1.710yrs,16545.89,24.99,0.17,17648.35,25.62,0.14,17431.87,25.96,0.13,17419.59,26.22,0.12


In [15]:
cols = df_in_order.columns.tolist()
cols

['name',
 'Y1 effective survey area ',
 'median Y1 $i$-band coadded depth in effective survey area',
 'std in Y1 $i$-band coadded depth in effective survey area',
 'Y3 effective survey area ',
 'median Y3 $i$-band coadded depth in effective survey area',
 'std in Y3 $i$-band coadded depth in effective survey area',
 'Y6 effective survey area ',
 'median Y6 $i$-band coadded depth in effective survey area',
 'std in Y6 $i$-band coadded depth in effective survey area',
 'Y10 effective survey area ',
 'median Y10 $i$-band coadded depth in effective survey area',
 'std in Y10 $i$-band coadded depth in effective survey area']

#### realized that there's too much finagling when sharing data so its best to reorder columns to make sure the order of the columns is the same.

In [16]:
new_cols = ['name',
            'median Y1 $i$-band coadded depth in effective survey area',
            'median Y3 $i$-band coadded depth in effective survey area',
            'median Y6 $i$-band coadded depth in effective survey area',
            'median Y10 $i$-band coadded depth in effective survey area',
            'std in Y1 $i$-band coadded depth in effective survey area',
            'std in Y3 $i$-band coadded depth in effective survey area',
            'std in Y6 $i$-band coadded depth in effective survey area',
            'std in Y10 $i$-band coadded depth in effective survey area',
            'Y1 effective survey area ',
            'Y3 effective survey area ',
            'Y6 effective survey area ',
            'Y10 effective survey area ',]
new_cols

['name',
 'median Y1 $i$-band coadded depth in effective survey area',
 'median Y3 $i$-band coadded depth in effective survey area',
 'median Y6 $i$-band coadded depth in effective survey area',
 'median Y10 $i$-band coadded depth in effective survey area',
 'std in Y1 $i$-band coadded depth in effective survey area',
 'std in Y3 $i$-band coadded depth in effective survey area',
 'std in Y6 $i$-band coadded depth in effective survey area',
 'std in Y10 $i$-band coadded depth in effective survey area',
 'Y1 effective survey area ',
 'Y3 effective survey area ',
 'Y6 effective survey area ',
 'Y10 effective survey area ']

In [17]:
df_in_order = df_in_order[new_cols]
df_in_order

,name,median Y1 $i$-band coadded depth in effective survey area,median Y3 $i$-band coadded depth in effective survey area,median Y6 $i$-band coadded depth in effective survey area,median Y10 $i$-band coadded depth in effective survey area,std in Y1 $i$-band coadded depth in effective survey area,std in Y3 $i$-band coadded depth in effective survey area,std in Y6 $i$-band coadded depth in effective survey area,std in Y10 $i$-band coadded depth in effective survey area,Y1 effective survey area,Y3 effective survey area,Y6 effective survey area,Y10 effective survey area
0,baseline_nexp1_v1.7_10yrs,25.10,25.73,26.07,26.33,0.18,0.14,0.13,0.12,15205.27,15383.52,15312.39,15307.98
1,baseline_nexp2_v1.7_10yrs,25.03,25.66,26.00,26.26,0.17,0.13,0.12,0.11,14765.22,15275.67,15197.30,15179.99
2,u_long_ms_50_v1.7_10yrs,25.01,25.63,25.97,26.24,0.16,0.12,0.12,0.11,14369.76,15202.39,15129.26,15091.39
3,rolling_scale0.2_nslice2_v1.7_10yrs,25.03,25.66,26.00,26.26,0.17,0.13,0.12,0.12,14779.75,15303.84,15207.11,15191.37
4,rolling_scale1.0_nslice2_v1.7_10yrs,25.03,25.64,25.97,26.24,0.17,0.13,0.12,0.11,14779.75,15253.38,15126.22,15100.41
5,footprint_8_v1.710yrs,24.99,25.62,25.96,26.22,0.17,0.14,0.13,0.12,16545.89,17648.35,17431.87,17419.59


In [18]:
# save the metrics
fname = 'static_metrics_selected.csv'
df_in_order.to_csv('%s/%s' % (outdir, fname), index=False)

#### now read and save lss-fom and ngal

In [19]:
# clean up the give_list dataframe
for key in df_in_order.keys():
    if key.__contains__('Y'):
        df_in_order = df_in_order.drop(key, axis=1)
        
df_in_order

,name
0,baseline_nexp1_v1.7_10yrs
1,baseline_nexp2_v1.7_10yrs
2,u_long_ms_50_v1.7_10yrs
3,rolling_scale0.2_nslice2_v1.7_10yrs
4,rolling_scale1.0_nslice2_v1.7_10yrs
5,footprint_8_v1.710yrs


#### read in the lss fom values

In [20]:
data_dir_base = '/global/cscratch1/sd/awan/lsst_output/post_wp_output_v1.7_-0.1cuts/lss_fom/'

In [21]:
data_dir = '%s/os_bias_overplots/' % data_dir_base
[f for f in os.listdir(data_dir) if f.__contains__('fomdata')] 

['2021-03-08_fomdata_Y1_i<23.95_snfom_egfootprint_74cadences_0.66<z<1.0_th-r<24.35_100<ell<300_logtcuts.csv',
 '2021-03-08_fomdata_Y10_i<25.2_snfom_egfootprint_74cadences_0.66<z<1.0_th-r<25.60_100<ell<300_logtcuts.csv']

In [22]:
fname = [f for f in os.listdir(data_dir) if f.__contains__('fomdata') if f.__contains__('_Y1_')][0]
print(fname)
data_file = '%s/%s' % (data_dir, fname)
lss_fom = pd.read_csv(data_file)

# find the indices to get the order
ind = []
for db in df_in_order['name']:
    ind_temp = np.where( lss_fom['dbname'].values == db.split('.db')[0] )[0]

    if len(ind_temp) == 0:
        print('%s not found in data dict' % db)
        #ind_temp = np.array([ 0 ])
        
    if ind_temp in ind:
        print('%s ind already in. %s' % (ind_temp, db))
    ind +=  list( ind_temp )
    
ind = np.array(ind)
print(ind)

df_in_order['Y1 lss fom'] = lss_fom['FoM'].values[ind]

2021-03-08_fomdata_Y1_i<23.95_snfom_egfootprint_74cadences_0.66<z<1.0_th-r<24.35_100<ell<300_logtcuts.csv
[32 64 62 35 65 54]


In [23]:
fname = [f for f in os.listdir(data_dir) if f.__contains__('fomdata') if f.__contains__('_Y10_')][0]
print(fname)
data_file = '%s/%s' % (data_dir, fname)
lss_fom = pd.read_csv(data_file)

# find the indices to get the order
ind = []
for db in df_in_order['name']:
    ind_temp = np.where( lss_fom['dbname'].values == db.split('.db')[0] )[0]

    if len(ind_temp) == 0:
        print('%s not found in data dict' % db)
        #ind_temp = np.array([ 0 ])
        
    if ind_temp in ind:
        print('%s ind already in. %s' % (ind_temp, db))
    ind +=  list( ind_temp )
    
ind = np.array(ind)
print(ind)

df_in_order['Y10 lss fom'] = lss_fom['FoM'].values[ind]

2021-03-08_fomdata_Y10_i<25.2_snfom_egfootprint_74cadences_0.66<z<1.0_th-r<25.60_100<ell<300_logtcuts.csv
[32 64 62 35 65 54]


In [24]:
df_in_order


,name,Y1 lss fom,Y10 lss fom
0,baseline_nexp1_v1.7_10yrs,0.933107,0.979715
1,baseline_nexp2_v1.7_10yrs,0.910802,0.975058
2,u_long_ms_50_v1.7_10yrs,0.930939,0.976924
3,rolling_scale0.2_nslice2_v1.7_10yrs,0.904981,0.974826
4,rolling_scale1.0_nslice2_v1.7_10yrs,0.904981,0.972155
5,footprint_8_v1.710yrs,0.931173,1.041267


#### now read in the ngal values

In [25]:
ngal_data_dir = data_dir_base

ngal_data = {}
for yr in [1, 10]:
    ngal_data[yr] =  np.zeros(len(df_in_order.name))
    folders = [f for f in os.listdir( ngal_data_dir ) if f.__contains__('_Y%s_' % yr) and f.__contains__('i<') ]
    for i, folder in enumerate( folders ):
        out = pd.read_csv('%s/%s/ngal-final_total/ngal_NoDither.txt' % (ngal_data_dir, folder), header=None, delimiter=' ') 
        dbname, ngal = out[0].values[0], out[1].values[0]
        ind = np.where( df_in_order['name'].values == '%s' % dbname)[0]
        ngal_data[yr][ind] = ngal
    print('read in %s folders for Y%s' % (i+1, yr) )

read in 74 folders for Y1
read in 74 folders for Y10


In [26]:
for yr in ngal_data:
    df_in_order['Y%s Ngal (0.66<z<1.0)' % yr] = ngal_data[yr]

In [27]:
df_in_order

,name,Y1 lss fom,Y10 lss fom,Y1 Ngal (0.66<z<1.0),Y10 Ngal (0.66<z<1.0)
0,baseline_nexp1_v1.7_10yrs,0.933107,0.979715,444000000.0,9.590000e+08
1,baseline_nexp2_v1.7_10yrs,0.910802,0.975058,431000000.0,9.490000e+08
2,u_long_ms_50_v1.7_10yrs,0.930939,0.976924,419000000.0,9.420000e+08
3,rolling_scale0.2_nslice2_v1.7_10yrs,0.904981,0.974826,431000000.0,9.500000e+08
4,rolling_scale1.0_nslice2_v1.7_10yrs,0.904981,0.972155,431000000.0,9.430000e+08
5,footprint_8_v1.710yrs,0.931173,1.041267,482000000.0,1.090000e+09


In [28]:
fname = 'lss_metrics_selected.csv'
df_in_order.to_csv('%s/%s' % (outdir, fname), index=False)